In [1]:
# Import necessary modules

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from random import randint, shuffle
import random
import re
#from libvoikko import Voikko
#import libvoikko
import pandas as pd # library for data analysis
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents

import libvoikko
from libvoikko import Voikko

Voikko.setLibrarySearchPath(r"C:\Users\rache\Downloads\data_science\finnish-app\Voikko")

v = libvoikko.Voikko("fi", r"C:\Users\rache\Downloads\data_science\finnish-app\Voikko\dict")

In [19]:
yle_url = 'https://yle.fi/uutiset/osasto/selkouutiset/'

response=requests.get(yle_url)

soup = BeautifulSoup(response.text, 'html.parser')

results = soup.find_all("div", {"class": "text"})

yle_articles = results[1].findAll(text=True)

h3s = results[1].findAll('h3')
headlines = []

for h3 in h3s:
    temp = str(h3)
    temp = temp.replace('<h3>', '')
    temp = temp.replace('</h3>', '')
    headlines.append(temp)
    del temp

unwanted_text = ['+ ', '–', '\n', '–\n', 'TV:n selkouutisilla on kesätauko. ', \
                      '(',')', ')\n', ';', '-"-\n', '; ']

unwanted_text = unwanted_text + headlines

article_list = [char for char in yle_articles if char not in unwanted_text]

article_list = [sent for sent in article_list if not '\n' in sent]

article_list = [sent_tokenize(sent) for sent in article_list]

article_list = [sent for sublist in article_list for sent in sublist]

article_list = [sent for sent in article_list if len(word_tokenize(sent))>1 ]

print(article_list)

['Afganistanissa Kabulin lentokentän lähellä on räjähtänyt.', 'Räjähdys oli iso.', 'Räjähdys oli todennäköisesti terrori-isku.', 'Useita ihmisiä kuoli räjähdyksessä.', 'Puolustusministeriö kertoo, että suomalaisia ei kuollut tai loukkaantunut.', 'Useat maat varoittivat jo aikaisemmin, että Kabulin lentokentälle suunnitellaan terrori-iskuja.', 'Lentokentän lähellä on paljon ihmisiä, jotka yrittävät päästä pois Afganistanista.', 'Evakuoinnit Afganistanista', 'Kabulista haettiin keskiviikkona Suomeen 51 ihmistä.', 'Afganistanista on evakuoitu Suomeen nyt yhteensä lähes 340 ihmistä, kertoo ulkoministeriö.', 'Ylen tiedot kertovat, että Suomi lopettaa evakuoinnit Afganistanista perjantaina.', 'Moni maa on jo lopettanut ihmisten evakuoinnin.', 'Uusia koronatartuntoja on Suomessa vähemmän kuin aikaisemmin.', 'Viranomainen kertoo, että uudet tartunnat ovat vähentyneet tällä viikolla lähes kaiken ikäisillä ihmisillä.', 'Viime viikolla uusia koronatartuntoja oli noin 800 vähemmän kuin viikkoa aik

In [27]:
pattern = RegexpTokenizer(r'\w+')

# Split into sentences

num = randint(0, len(article_list)-1)

sentence = article_list[num]

# Tokenize sentence, remove punctuation, numbers, and capitalized words (to avoid proper nouns).

tokenized_sent = pattern.tokenize(sentence)

tokenized_sent = [word for word in tokenized_sent if re.sub(r'[0-9]', '', word)]

tokenized_sent = [word for word in tokenized_sent if not word[0].isupper()]

num = randint(0, len(tokenized_sent)-1)

word = tokenized_sent[num]

print(word)

pystyvät


In [61]:
lemmatized = v.analyze(word)[0]

word_baseform = lemmatized['BASEFORM']
word_class = lemmatized['CLASS']

print(word_baseform, word_class)

0


In [53]:
#Extract the base form
#word_baseform = voikko_dict[0]['BASEFORM']

############
word = 'ssä'
lemmatized = v.analyze(word)[0]

word_baseform = lemmatized['BASEFORM']
word_class = lemmatized['CLASS']

print(word_baseform, word_class)

########

cases = []

wikiurl = ''

if word_class == 'teonsana':
    wikiurl = 'https://fi.wiktionary.org/wiki/Liite:Verbitaivutus/suomi/'+ word_baseform
elif word_class == 'nimisana':
    wikiurl = 'https://fi.wiktionary.org/wiki/'+ word_baseform + '#Taivutus'
elif word_class == 'laatusana':
    wikiurl = 'https://fi.wiktionary.org/wiki/Liite:Adjektiivitaivutus/suomi/'+ word_baseform
elif word_class == 'asemosana':
    wikiurl = 'https://fi.wiktionary.org/wiki/'+ word_baseform
else: wikiurl = 'https://fi.wiktionary.org'
    
# get the response in the form of html


table_class="wikitable sortable jquery-tablesorter"

response=requests.get(wikiurl)

# parse data from the html into a beautifulsoup object

soup = BeautifulSoup(response.text, 'html.parser')

wikitable=soup.find('table',{'class':"wikitable"})

if wikitable is None:
    table_contents = ''
else: table_contents = wikitable.findAll(text=True)
    
unwanted_chars = [word, '+ ', '–', '\n', '–\n', 'Taivutus', 'Taivutus\n', 'sijamuoto', 'nominatiivi', 'genetiivi', 'partitiivi', \
                  'akkusatiivi', 'sisäpaikallissijat', 'inessiivi', 'elatiivi', 'illatiivi', 'ulkopaikallissijat', \
                  'adessiivi', 'ablatiivi', 'allatiivi', 'essiivi', 'translatiivi', 'abessiivi', 'instruktiivi', \
                  'komitatiivi', 'positiivi', 'sijamuoto', 'yksikkö', 'monikko', 'kieliopilliset sijamuodot\n', \
                  'sisäpaikallissijat\n', 'ulkopaikallissijat\n', 'muut sijamuodot\n', 'omistusliite', 'monikko\n', \
                  'Positiivi', 'Komparatiivi', 'Superlatiivi', 'muut\n','(',')', ')\n', ';', 'Indikatiivi\n', 'preesens\n', \
                  'perfekti\n', 'persoona\n', 'myönteinen\n', 'kielteinen\n', 'persoona\n', 'myönteinen\n', 'kielteinen\n', 'minä\n', \
                  'sinä\n','hän\n','me\n','-"-\n','te', 'Te\n','he\n','passiivi','imperfekti\n','pluskvamperfekti\n', '; ']

negatives = ['en ', 'olen ', 'en ole ', 'et ', 'olet ', 'et ole ', 'ei ',  'on ', 'ei ole ', 'emme ', 'olemme ', 'emme ole ',\
             'ette ', 'olette ',  'ette ole ', 'ette ole ', 'eivät ', 'ovat ', 'eivät ole ', 'olin ', 'en ollut ', 'olit ', \
             'et ollut ', 'oli ',  'ei ollut ', 'olimme ', 'emme olleet ', 'olitte ', 'ette olleet ', 'ette olleet ',  'olivat ', \
             'eivät olleet ']

if len(table_contents) > 0:
    cases = [char for char in table_contents if char not in unwanted_chars]
    
    cases = [''.join(e for e in case if e.isalnum()) for case in cases]

    cases = [char for char in cases if char not in negatives]

    cases = list(dict.fromkeys(cases))

print(wikiurl)
print(cases)

IndexError: list index out of range

In [ ]:
#set options from selected word and randomly selected cases

fill_in_blank = sentence.replace(' '+word, '_______')

num = randint(0, len(cases)-1)
option_2 = cases[num]

num = randint(0, len(cases)-1)
option_3 = cases[num]

options = [word, option_2, option_3]

options = random.sample(options, len(options))

#print(sentence)
print(fill_in_blank)
print(options)

In [ ]:
#Combine into function

def practice():

    yle_url = 'https://yle.fi/uutiset/osasto/selkouutiset/'

    response=requests.get(yle_url)

    soup = BeautifulSoup(response.text, 'html.parser')

    results = soup.find("div", {"class": "text"})

    yle_articles = results.findAll(text=True)

    #removing headlines and cleaning characters from scraped text
    h3s = results.findAll('h3')
    headlines = []

    for h3 in h3s:
        temp = str(h3)
        temp = temp.replace('<h3>', '')
        temp = temp.replace('</h3>', '')
        headlines.append(temp)
        del temp

    unwanted_text = ['+ ', '–', '\n', '–\n', 'TV:n selkouutisilla on kesätauko. ', \
                      '(',')', ')\n', ';', '-"-\n', '; ']

    unwanted_text = unwanted_text + headlines

    article_list = [char for char in yle_articles if char not in unwanted_text]

    article_list = [sent for sent in article_list if not '\n' in sent]
    
    article_list = [sent_tokenize(sent) for sent in article_list]

    article_list = [item for sublist in article_list for item in sublist]

    print(article_list)

    pattern = RegexpTokenizer(r'\w+')

    # Split into sentences

    num = randint(0, len(article_list)-1)

    sentence = article_list[num]

    #print('Sent: '+ sentence)

    # Tokenize sentence, remove punctuation, numbers, and capitalized words (to avoid proper nouns).

    tokenized_sent = pattern.tokenize(sentence)

    tokenized_sent = [word for word in tokenized_sent if re.sub(r'[0-9]', '', word)]

    tokenized_sent = [word for word in tokenized_sent if not word[0].isupper()]

    num = randint(0, len(tokenized_sent)-1)

    word = tokenized_sent[num]

    fill_in_blank = sentence.replace(' '+word, ' _____ ')

    #print('Word1: '+word)
    #print("blank1: "+fill_in_blank)

    if len(v.analyze(word) > 0)
        lemmatized = v.analyze(word)[0]

        word_baseform = lemmatized['BASEFORM']
        word_class = lemmatized['CLASS']

        #print('Lemm: '+word_baseform, word_class)

        #Extract the base form
        #word_baseform = voikko_dict[0]['BASEFORM']
        cases = []

        wikiurl = ''

        if word_class == 'teonsana':
            wikiurl = 'https://fi.wiktionary.org/wiki/Liite:Verbitaivutus/suomi/'+ word_baseform
        elif word_class == 'nimisana':
            wikiurl = 'https://fi.wiktionary.org/wiki/'+ word_baseform + '#Taivutus'
        elif word_class == 'laatusana':
            wikiurl = 'https://fi.wiktionary.org/wiki/Liite:Adjektiivitaivutus/suomi/'+ word_baseform
        elif word_class == 'asemosana':
            wikiurl = 'https://fi.wiktionary.org/wiki/'+ word_baseform
        else: wikiurl = 'https://fi.wiktionary.org'

        # get the response in the form of html


        table_class="wikitable sortable jquery-tablesorter"

        response=requests.get(wikiurl)

        # parse data from the html into a beautifulsoup object

        soup = BeautifulSoup(response.text, 'html.parser')

        wikitable=soup.find('table',{'class':"wikitable"})

        if wikitable is None:
            table_contents = ''
        else: table_contents = wikitable.findAll(text=True)

        unwanted_chars = [word, '+ ', '–', '\n', '–\n', 'Taivutus\n', 'sijamuoto', 'nominatiivi', 'genetiivi', 'partitiivi', \
                          'akkusatiivi', 'sisäpaikallissijat', 'inessiivi', 'elatiivi', 'illatiivi', 'ulkopaikallissijat', \
                          'adessiivi', 'ablatiivi', 'allatiivi', 'essiivi', 'translatiivi', 'abessiivi', 'instruktiivi', \
                          'komitatiivi', 'positiivi', 'sijamuoto', 'yksikkö', 'monikko', 'kieliopilliset sijamuodot\n', \
                          'sisäpaikallissijat\n', 'ulkopaikallissijat\n', 'muut sijamuodot\n', 'omistusliite', 'monikko\n', \
                          'Positiivi', 'Komparatiivi', 'Superlatiivi', 'muut\n','(',')', ')\n', ';', 'Indikatiivi\n', 'preesens\n', \
                          'perfekti\n', 'persoona\n', 'myönteinen\n', 'kielteinen\n', 'persoona\n', 'myönteinen\n', 'kielteinen\n', 'minä\n', \
                          'sinä\n','hän\n','me\n','-"-\n','te', 'Te\n','he\n','passiivi','imperfekti\n','pluskvamperfekti\n', '; ']

        negatives = ['en ', 'olen ', 'en ole ', 'et ', 'olet ', 'et ole ', 'ei ',  'on ', 'ei ole ', 'emme ', 'olemme ', 'emme ole ',\
                     'ette ', 'olette ',  'ette ole ', 'ette ole ', 'eivät ', 'ovat ', 'eivät ole ', 'olin ', 'en ollut ', 'olit ', \
                     'et ollut ', 'oli ',  'ei ollut ', 'olimme ', 'emme olleet ', 'olitte ', 'ette olleet ', 'ette olleet ',  'olivat ', \
                     'eivät olleet ']

        if len(table_contents) > 0:
            cases = [char for char in table_contents if char not in unwanted_chars]

            cases = [char for char in cases if char not in negatives]

            cases = list(dict.fromkeys(cases))

        #print(wikiurl)
        #print(cases)

        #set options from selected word and randomly selected cases
        if len(cases) > 0:
            num = randint(0, len(cases)-1)
            option_2 = cases[num].replace('-', '').replace('(', '').replace(')', '')

            num = randint(0, len(cases)-1)
            option_3 = cases[num].replace('-', '').replace('(', '').replace(')', '')

            options = [word, option_2, option_3]

            options = random.sample(options, len(options))

            return fill_in_blank, options, word
        else: return practice()
    else: return practice()

practice()

#ERRORS

- errors on loading
- tokanizes multiple sentences at once
- HEROKU cannot load voikko libraries
- special characters (e.g. 44-vuotias)
- loads incomplete sentences
- yle_articles loads 2 arrays?


In [2]:
#WORKING OUT ERRORS

def practice():

    #yle_url = 'https://yle.fi/uutiset/osasto/selkouutiset/'
    yle_url = 'https://yle.fi/uutiset/osasto/selkouutiset/lauantai_2182021/12067864'

    response=requests.get(yle_url)

    soup = BeautifulSoup(response.text, 'html.parser')

    results = soup.find("div", {"class": "text"})

    yle_articles = results.findAll(text=True)

    print('YLE_ARTICLES: ', yle_articles)
    
    #removing headlines and cleaning characters from scraped text
    h3s = results.findAll('h3')
    headlines = []

    for h3 in h3s:
        temp = str(h3)
        temp = temp.replace('<h3>', '')
        temp = temp.replace('</h3>', '')
        headlines.append(temp)
        del temp

    unwanted_text = ['+ ', '–', '\n', '–\n', \
                      '(',')', ')\n', ';', '-"-\n', '; ']

    unwanted_text = unwanted_text + headlines

    article_list = [char for char in yle_articles if char not in unwanted_text]

    article_list = [sent for sent in article_list if not '\n' in sent]
    
    article_list = [sent_tokenize(sent) for sent in article_list]

    article_list = [sent for sublist in article_list for sent in sublist]
    
    article_list = [sent for sent in article_list if len(word_tokenize(sent))>1 ]

    print('ARTICLES_LIST: ', article_list)

    pattern = RegexpTokenizer(r'\w+')

    # Split into sentences

    num = randint(0, len(article_list)-1)

    sentence = article_list[num]

    print('SENT: '+ sentence)

    # Tokenize sentence, remove punctuation, numbers, and capitalized words (to avoid proper nouns).

    tokenized_sent = pattern.tokenize(sentence)

    tokenized_sent = [word for word in tokenized_sent if re.sub(r'[0-9]', '', word)]

    tokenized_sent = [word for word in tokenized_sent if not word[0].isupper()]
    
    print('TOK_SENT: ', tokenized_sent)

    num = randint(0, len(tokenized_sent)-1)

    word = tokenized_sent[num]

    fill_in_blank = sentence.replace(' '+word, ' _____ ')

    print("SENTENCE: ", sentence)
    #print("blank1: "+fill_in_blank)
    
    if len(v.analyze(word)) > 0:

        lemmatized = v.analyze(word)[0]

        word_baseform = lemmatized['BASEFORM']
        word_class = lemmatized['CLASS']

        print('LEMM: '+word_baseform, word_class)

        #Extract the base form
        #word_baseform = voikko_dict[0]['BASEFORM']
        cases = []

        wikiurl = ''

        if word_class == 'teonsana':
            wikiurl = 'https://fi.wiktionary.org/wiki/Liite:Verbitaivutus/suomi/'+ word_baseform
        elif word_class == 'nimisana':
            wikiurl = 'https://fi.wiktionary.org/wiki/'+ word_baseform + '#Taivutus'
        elif word_class == 'laatusana':
            wikiurl = 'https://fi.wiktionary.org/wiki/Liite:Adjektiivitaivutus/suomi/'+ word_baseform
        elif word_class == 'asemosana':
            wikiurl = 'https://fi.wiktionary.org/wiki/'+ word_baseform
        else: wikiurl = 'https://fi.wiktionary.org'

        # get the response in the form of html


        table_class="wikitable sortable jquery-tablesorter"

        response=requests.get(wikiurl)

        # parse data from the html into a beautifulsoup object

        soup = BeautifulSoup(response.text, 'html.parser')

        wikitable=soup.find('table',{'class':"wikitable"})

        if wikitable is None:
            table_contents = ''
        else: table_contents = wikitable.findAll(text=True)

        unwanted_chars = [word, '+ ', '–', '\n', '–\n', 'Taivutus\n', 'sijamuoto', 'nominatiivi', 'genetiivi', 'partitiivi', \
                          'akkusatiivi', 'sisäpaikallissijat', 'inessiivi', 'elatiivi', 'illatiivi', 'ulkopaikallissijat', \
                          'adessiivi', 'ablatiivi', 'allatiivi', 'essiivi', 'translatiivi', 'abessiivi', 'instruktiivi', \
                          'komitatiivi', 'positiivi', 'sijamuoto', 'yksikkö', 'monikko', 'kieliopilliset sijamuodot\n', \
                          'sisäpaikallissijat\n', 'ulkopaikallissijat\n', 'muut sijamuodot\n', 'omistusliite', 'monikko\n', \
                          'Positiivi', 'Komparatiivi', 'Superlatiivi', 'muut\n','(',')', ')\n', ';', 'Indikatiivi\n', 'preesens\n', \
                          'perfekti\n', 'persoona\n', 'myönteinen\n', 'kielteinen\n', 'persoona\n', 'myönteinen\n', 'kielteinen\n', 'minä\n', \
                          'sinä\n','hän\n','me\n','-"-\n','te', 'Te\n','he\n','passiivi','imperfekti\n','pluskvamperfekti\n', '; ']

        negatives = ['en ', 'olen ', 'en ole ', 'et ', 'olet ', 'et ole ', 'ei ',  'on ', 'ei ole ', 'emme ', 'olemme ', 'emme ole ',\
                     'ette ', 'olette ',  'ette ole ', 'ette ole ', 'eivät ', 'ovat ', 'eivät ole ', 'olin ', 'en ollut ', 'olit ', \
                     'et ollut ', 'oli ',  'ei ollut ', 'olimme ', 'emme olleet ', 'olitte ', 'ette olleet ', 'ette olleet ',  'olivat ', \
                     'eivät olleet ']

        if len(table_contents) > 0:
            cases = [char for char in table_contents if char not in unwanted_chars]

            cases = [char for char in cases if char not in negatives]

            cases = [''.join(e for e in case if e.isalnum()) for case in cases]

            cases = list(dict.fromkeys(cases))

        #set options from selected word and randomly selected cases
        if len(cases) > 0:
            num = randint(0, len(cases)-1)
            option_2 = cases[num].replace('-', '').replace('(', '').replace(')', '')

            num = randint(0, len(cases)-1)
            option_3 = cases[num].replace('-', '').replace('(', '').replace(')', '')

            options = [word, option_2, option_3]

            options = random.sample(options, len(options))

            return fill_in_blank, options, word
        else: return practice()
    else: return practice()

practice()

YLE_ARTICLES:  ['\n', '\n', 'TV:n selkouutisilla on kesätauko. Selkouutiset jatkuvat TV:ssä taas maanantaina 23.8.', 'Radion selkouutiset tehdään joka päivä myös kesällä.', 'Suomalaiset Kabulissa', '\n', '\n', '\n', '\n', '\n', '\n', '\nUlkoministeri Pekka Haavisto.\n', '\n', '\nKuva:\nLauri Heino / Lehtikuva\n', '\n', '\n', '\n', 'Suomalaiset sotilaat ovat saapuneet Afganistaniin Kabulin lentokentälle. Sotilaat suojaavat suomalaisten evakuointia Kabulissa. Sitä ei ole kerrottu, kuinka monta sotilasta Kabuliin meni. ', 'Suomi on evakuoinut Afganistanista jo 50 ihmistä. Ulkoministeri ', 'Pekka Haavisto ', 'sanoo, että Suomi on evakuoinut Suomen lähetystön henkilökuntaa ja ihmisiä, jotka ovat tehneet töitä Suomelle. ', 'Suomi jatkaa evakuointeja. Suomella on Kabulin lentokentällä nyt 5 suomalaista virkamiestä auttamassa evakuointia. ', 'Rauhallisesti koronarokotuksen jälkeen', '\n', '\n', '\n', '\n', '\n', '\n', '\nPuolustusvoimissa ei tehdä raskaita töitä heti rokotuksen jälkeen.\n', '\

YLE_ARTICLES:  ['\n', '\n', 'TV:n selkouutisilla on kesätauko. Selkouutiset jatkuvat TV:ssä taas maanantaina 23.8.', 'Radion selkouutiset tehdään joka päivä myös kesällä.', 'Suomalaiset Kabulissa', '\n', '\n', '\n', '\n', '\n', '\n', '\nUlkoministeri Pekka Haavisto.\n', '\n', '\nKuva:\nLauri Heino / Lehtikuva\n', '\n', '\n', '\n', 'Suomalaiset sotilaat ovat saapuneet Afganistaniin Kabulin lentokentälle. Sotilaat suojaavat suomalaisten evakuointia Kabulissa. Sitä ei ole kerrottu, kuinka monta sotilasta Kabuliin meni. ', 'Suomi on evakuoinut Afganistanista jo 50 ihmistä. Ulkoministeri ', 'Pekka Haavisto ', 'sanoo, että Suomi on evakuoinut Suomen lähetystön henkilökuntaa ja ihmisiä, jotka ovat tehneet töitä Suomelle. ', 'Suomi jatkaa evakuointeja. Suomella on Kabulin lentokentällä nyt 5 suomalaista virkamiestä auttamassa evakuointia. ', 'Rauhallisesti koronarokotuksen jälkeen', '\n', '\n', '\n', '\n', '\n', '\n', '\nPuolustusvoimissa ei tehdä raskaita töitä heti rokotuksen jälkeen.\n', '\

('Siksi Suomen armeijassa sotilaat vapautetaan raskaista tehtävistä 3 päivän _____  rokotuksen jälkeen.',
 ['ajatta', 'aikoine', 'ajaksi'],
 'ajaksi')

In [30]:
#ERROR prone example:

yle_articles_test = ['\n', '\n', 'TV:n selkouutisilla on kesätauko. Selkouutiset nähdään TV:ssä taas maanantaina 23.8. ', \
                     'Radion selkouutiset tehdään joka päivä myös kesällä.', 'Perussuomalaiset', '\n', '\n', '\n', '\n', '\n', \
                     '\n', '\nRiikka Purra Perussuomalaisten puoluekokouksessa Seinäjoella.\n', '\n', \
                     '\nKuva:\nMarkku Ulander / Lehtikuva\n', '\n', '\n', '\n', \
                     'Perussuomalaiset-puolue on valinnut uuden puheenjohtajan. Perussuomalaisten uusi puheenjohtaja on ', \
                     'Riikka Purra', '. ', \
                     'Hän sai puoluekokouksen äänestyksessä 774 ääntä eli paljon enemmän ääniä kuin muut ehdokkaat. Toiseksi eniten ääniä sai Sakari Puisto, joka sai 252 ääntä.', \
                     'Perussuomalaisten uusi puheenjohtaja Riikka Purra sanoo, että hän haluaa kirkastaa puolueen linjaa maahanmuutosta. Purra sanoo, että perussuomalaisten pitää esimerkiksi kertoa paremmin, miksi puolue suhtautuu kriittisesti maahanmuuttoon. ', \
                     'Riikka Purra on 44-vuotias. Hän on myös perussuomalaisten kansanedustaja. ', 'Afganistan', '\n', '\n', \
                     '\n', '\n', '\n', '\n', '\nYK:n pääsihteeri Antonio Guterres.\n', '\n', '\nKuva:\nMartial Trezzini / AOP\n',\
                     '\n', '\n', '\n', 'Afganistanissa taistelut kiihtyvät. Taliban-liike on vallannut lisää alueita.', \
                     'Lauantaina aamulla Taliban aloitti ison hyökkäyksen Mazar-i-Sharifin kaupunkiin Afganistanin pohjoisosassa. Uutistoimisto AP kertoi lauantaina päivällä, että taistelut lähestyvät myös pääkaupunkia Kabulia.',\
                     'YK:n pääsihteeri ', 'Antonio Guterres ', \
                     'pyytää, että Taliban lopettaa hyökkäykset. Guterres sanoo, että Afganistan voi suistua kaaokseen.', \
                     'Koronan kotitestit', '\n', '\n', '\n', '\n', '\n', '\n', \
                     '\nKoronan kotitestipakkauksia. Testi, joka tehdään kotona, ei anna virallista tulosta.\n', '\n', \
                     '\nKuva:\nMinna Rosvall / Yle\n', '\n', '\n', '\n', \
                     'Koronaviruksen kotitestien myynti on lisääntynyt Suomessa. Kotitestejä myydään esimerkiksi ruokakaupoissa. Testit ovat loppuneet joistakin kaupoista. ', \
                     'Kaupat arvioivat, että koronan kotitestejä on ostettu paljon, koska koronatilanne on taas huonontunut. Ihmiset ovat myös huomanneet, että kotitestejä myydään kaupoissa. ', \
                     'Koronaviruksen kotitestistä ei kuitenkaan saa virallista testitulosta. ', 'Lämpö maapallolla', '\n', '\n', '\n', '\n', '\n', \
                     '\n', '\nMyös Suomessa oli heinäkuussa kuumaa ja kuivaa. Monet kasvit kellastuivat. ', '\n', '\nKuva:\nJouni Tanninen / Yle\n',\
                     '\n', '\n', '\n', \
                     'Heinäkuu oli tosi kuuma maapallolla. USA:n säätutkijat sanovat, että heinäkuu oli kuumin kuukausi, mitä maapallolla on mitattu. Mittauksia on tehty 142 vuotta.',\
                     'USA:n säätutkijoiden arvio eroaa eurooppalaisten tutkijoiden tuloksista. ', \
                     'Eurooppalaiset säätutkijat sanovat, että heinäkuu oli kolmanneksi kuumin, mitä maapallolla on mitattu.', \
                     'Sää', '\n', '\n','\n', '\n', '\n', '\n', \
                     '\nSunnuntaina tulee sateita eri puolilla Suomea.\n', \
                     '\n', '\n', '\n', \
                     'Sää on sunnuntaina puolipilvin tai pilvinen. Sadetta tai sadekuuroja tulee eri puolilla maata. Paikoin on myös ukkosia. Lämpötila on 15 ja 19 asteen välillä. ','\n'] 

article_list_test = ['TV:n selkouutisilla on kesätauko. Selkouutiset nähdään TV:ssä taas maanantaina 23.8. ', \
        'Radion selkouutiset tehdään joka päivä myös kesällä.', \
        'Perussuomalaiset-puolue on valinnut uuden puheenjohtajan. Perussuomalaisten uusi puheenjohtaja on ', \
        'Riikka Purra', '. ', \
        'Hän sai puoluekokouksen äänestyksessä 774 ääntä eli paljon enemmän ääniä kuin muut ehdokkaat. Toiseksi eniten ääniä sai Sakari Puisto, joka sai 252 ääntä.', \
        'Perussuomalaisten uusi puheenjohtaja Riikka Purra sanoo, että hän haluaa kirkastaa puolueen linjaa maahanmuutosta. Purra sanoo, että perussuomalaisten pitää esimerkiksi kertoa paremmin, miksi puolue suhtautuu kriittisesti maahanmuuttoon. ', \
        'Riikka Purra on 44-vuotias. Hän on myös perussuomalaisten kansanedustaja. ', \
        'Afganistanissa taistelut kiihtyvät. Taliban-liike on vallannut lisää alueita.', \
        'Lauantaina aamulla Taliban aloitti ison hyökkäyksen Mazar-i-Sharifin kaupunkiin Afganistanin pohjoisosassa. Uutistoimisto AP kertoi lauantaina päivällä, että taistelut lähestyvät myös pääkaupunkia Kabulia.', \
        'YK:n pääsihteeri ', 'Antonio Guterres ', 'pyytää, että Taliban lopettaa hyökkäykset. Guterres sanoo, että Afganistan voi suistua kaaokseen.', \
        'Koronaviruksen kotitestien myynti on lisääntynyt Suomessa. Kotitestejä myydään esimerkiksi ruokakaupoissa. Testit ovat loppuneet joistakin kaupoista. ', \
        'Kaupat arvioivat, että koronan kotitestejä on ostettu paljon, koska koronatilanne on taas huonontunut. Ihmiset ovat myös huomanneet, että kotitestejä myydään kaupoissa. ', \
        'Koronaviruksen kotitestistä ei kuitenkaan saa virallista testitulosta. ', \
        'Heinäkuu oli tosi kuuma maapallolla. USA:n säätutkijat sanovat, että heinäkuu oli kuumin kuukausi, mitä maapallolla on mitattu. Mittauksia on tehty 142 vuotta.', \
        'USA:n säätutkijoiden arvio eroaa eurooppalaisten tutkijoiden tuloksista. ', \
        'Eurooppalaiset säätutkijat sanovat, että heinäkuu oli kolmanneksi kuumin, mitä maapallolla on mitattu.', \
        'Sää on sunnuntaina puolipilvin tai pilvinen. Sadetta tai sadekuuroja tulee eri puolilla maata. Paikoin on myös ukkosia. Lämpötila on 15 ja 19 asteen välillä. ']

yle_test1 = ['TV:n selkouutisilla on kesätauko.', 'Selkouutiset nähdään TV:ssä taas maanantaina 23.8.', 'Radion selkouutiset tehdään joka päivä myös kesällä.', 'Suomi on lähettänyt 2 virkamiestä Kabulin lentokentälle Afganistaniin.', 'Virkamiehet auttavat, kun ihmisiä haetaan Afganistanista Suomeen.', 'Asiasta kertoo ulkoministeri Pekka Haavisto.', 'Haavisto sanoo, että lähes 70 Suomen kansalaista on pyytänyt apua.', 'He haluavat Suomeen Afganistanista.', 'Suomi yrittää tuoda Suomeen myös Afganistanin Suomen lähetystön työntekijät.', 'Mielenosoituksia Afganistanissa', 'Afganistanissa on osoitettu mieltä Talibania vastaan.', 'Mediatiedot kertovat, että talibanit ovat ampuneet mielenosoittajia kohti Jalalabadin kaupungissa.', 'Ainakin 2 ihmistä on kuollut ja noin 10 ihmistä on loukkaantunut.', 'Koronaluvut', 'Suomessa on 673 uutta koronatartuntaa.', 'THL kertoo, että uusia koronakuolemia on 2.', 'Sairaalassa on nyt 106 ihmistä.', 'Heistä 29 saa tehohoitoa.', '2 viikon aikana on ollut lähes 2700 koronatartuntaa enemmän kuin 2 viikkoa aikaisemmin.', 'Kokoomus koronarokotukset', 'Oppositiopuolue kokoomus ehdottaa hallitukselle, että koronatestejä tehdään jatkossa myös apteekeissa.', 'Kokoomus haluaa, että testauspaikkoja on tulevaisuudessa enemmän kuin nyt.', 'Testien pitää kokoomuksen mielestä myös olla halvempia kuin nyt.', 'Kokoomuksen ehdotuksesta kertoo puheenjohtaja Petteri Orpo.', 'Hän puhui kokoomuksen kesäkokouksessa Seinäjoella.', 'Orpon mielestä Suomen apteekit pystyvät tekemään koronatestejä.', 'Orpo sanoo, että muualla Euroopassa koronatestejä tehdään paljon enemmän kuin Suomessa.', 'Konkurssit', 'Yrityksiä menee konkurssiin vähemmän kuin aikaisemmin.', 'Tilastokeskus kertoo, että konkursseja oli alkuvuonna 1,6 prosenttia vähemmän kuin samaan aikaan viime vuonna.', 'Varsinkin ravintoloita ja hotelleja meni konkurssiin vähemmän kuin viime vuonna.', 'Ravintoloiden ja hotellien konkurssit vähentyivät yli 16 prosenttia.', 'Maataloudessa ja rakennusalalla konkursseja oli enemmän kuin aikaisemmin.']

In [18]:
voikko_sub = {'nimisana':['asia', 'aika', 'kuva'], 'teosana':['olla', 'antaa', 'tehdä'], 'laatusana':['nopea', 'eri', 'hyvä'] }

print(voikko_sub['nimisana'])

print(v.analyze('minä')[0]['CLASS'])

['asia', 'aika', 'kuva']
asemosana
